In [1]:
import tensorflow as tf
import numpy as np
import random
import pandas as pd

from zipfile import ZipFile

random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

In [2]:
train_df = pd.read_csv("Train.csv")
test_df = pd.read_csv("Test.csv")

# add extension to the file names
def addextension(nm):
    return nm+".jpg"

train_df["Image_ID"] = train_df["Image_ID"].apply(addextension)
test_df["Image_ID"] = test_df["Image_ID"].apply(addextension)

print(train_df.head())

# unzip Train_Images and Test_Images
def unzip(nm):
    with ZipFile(nm,"r") as zip:
        zip.extractall()

unzip("Train_Images.zip")
unzip("Test_Images.zip")

          Image_ID            class   xmin   ymin  width  height
0  ID_007FAIEI.jpg  fruit_woodiness   87.0   87.5  228.0   311.0
1  ID_00G8K1V3.jpg  fruit_brownspot   97.5   17.5  245.0   354.5
2  ID_00WROUT9.jpg  fruit_brownspot  156.5  209.5  248.0   302.5
3  ID_00ZJEEK3.jpg    fruit_healthy  125.0  193.0  254.5   217.0
4  ID_018UIENR.jpg  fruit_brownspot   79.5  232.5  233.5   182.0


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_gen = ImageDataGenerator(
    horizontal_flip = True,
    rotation_range = 10,
    zoom_range = 0.1,
    validation_split = 0.2,
)

train_ds = train_gen.flow_from_dataframe(
    directory = "Train_Images",
    dataframe = train_df,
    x_col = "Image_ID",
    y_col = "class",
    target_size = (256,256),
    batch_size = 32,
    class_mode = "categorical",
    shuffle = True,
    subset = "training",
)

val_ds = train_gen.flow_from_dataframe(
    directory = "Train_Images",
    dataframe = train_df,
    x_col = "Image_ID",
    y_col = "class",
    target_size = (256,256),
    batch_size = 32,
    class_mode = "categorical",
    shuffle = True,
    subset = "validation",
)

Found 3125 validated image filenames belonging to 3 classes.
Found 781 validated image filenames belonging to 3 classes.


### Model = Transfer_VGG16 (Transfer Learning - Fine Tuning)

In [4]:
rescale = tf.keras.applications.vgg16.preprocess_input
base_model = tf.keras.applications.VGG16(input_shape=(256,256,3),include_top=False,weights="imagenet")
base_model.trainable = True

fine_tune_at = 10

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

class Transfer_VGG16():
    def model(self,input):
        self.x = rescale(input)
        self.x = base_model(self.x,training=False)
        self.x = tf.keras.layers.GlobalAveragePooling2D()(self.x)
        self.x = tf.keras.layers.Flatten()(self.x)
        self.x = tf.keras.layers.Dense(128,activation="relu")(self.x)
        self.x = tf.keras.layers.Dropout(0.2,seed=42)(self.x)
        self.x = tf.keras.layers.Dense(64,activation="relu")(self.x)
        self.x = tf.keras.layers.Dropout(0.2,seed=42)(self.x)
        self.output = tf.keras.layers.Dense(3,activation="softmax")(self.x)
        self.model = tf.keras.Model(input,self.output,name="Transfer_VGG16")
        return self.model

TFVGG16 = Transfer_VGG16()
model = TFVGG16.model(tf.keras.Input(shape=(256,256,3)))
model.compile(tf.keras.optimizers.RMSprop(1e-5),tf.keras.losses.CategoricalCrossentropy(),metrics=["accuracy"])
model.summary()

Model: "Transfer_VGG16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
tf.__operators__.getitem (Sl (None, 256, 256, 3)       0         
_________________________________________________________________
tf.nn.bias_add (TFOpLambda)  (None, 256, 256, 3)       0         
_________________________________________________________________
vgg16 (Functional)           (None, 8, 8, 512)         14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 128)            

### Train and evaluate the model

In [5]:
if __name__=="__main__":
    checkpoint = tf.keras.callbacks.ModelCheckpoint("makerere.hdf5",save_weights_only=False,save_best_only=True,monitor="val_accuracy")
    model.fit(train_ds,epochs=10,validation_data=val_ds,callbacks=[checkpoint])
    best = tf.keras.models.load_model("makerere.hdf5")
    val_loss,val_acc = best.evaluate(val_ds)
    print("\nVal Accuracy: {:.2f} %".format(100*val_acc))
    print("Val Loss: {:.2f} %".format(100*val_loss))

Epoch 1/10
98/98 [==============================] - 87s 729ms/step - loss: 0.6209 - accuracy: 0.7824 - val_loss: 0.2420 - val_accuracy: 0.8963
Epoch 2/10
98/98 [==============================] - 57s 575ms/step - loss: 0.2093 - accuracy: 0.9280 - val_loss: 0.1169 - val_accuracy: 0.9565
Epoch 3/10
98/98 [==============================] - 58s 586ms/step - loss: 0.1336 - accuracy: 0.9562 - val_loss: 0.1092 - val_accuracy: 0.9654
Epoch 4/10
98/98 [==============================] - 58s 586ms/step - loss: 0.0757 - accuracy: 0.9763 - val_loss: 0.0817 - val_accuracy: 0.9706
Epoch 5/10
98/98 [==============================] - 57s 582ms/step - loss: 0.0707 - accuracy: 0.9779 - val_loss: 0.0874 - val_accuracy: 0.9680
Epoch 6/10
98/98 [==============================] - 60s 607ms/step - loss: 0.0523 - accuracy: 0.9834 - val_loss: 0.0826 - val_accuracy: 0.9706
Epoch 7/10
98/98 [==============================] - 60s 609ms/step - loss: 0.0364 - accuracy: 0.9904 - val_loss: 0.0597 - val_accuracy: 0.9808